In [15]:
import os
from api import API_KEYS

os.environ["OPENAI_API_KEY"] = API_KEYS["OPENAI"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = API_KEYS["HUGGINFACE"]

## API CALLING TEST

### 프롬포트 생성

- PromptTemplate은 LLM에 문장을 전달하기 전에 문장 구성을 편리하게 만들어주는 역할
- LLM이 어떤 문장을 만들어야 하는지를 알려주는 역할을 수행

In [ ]:
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 좋은 문구를 추천해줘?"

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="카메라")

'카메라를 홍보하기 위한 좋은 문구를 추천해줘?'